# Predicting Property Prices


## Import Libraries


In [ ]:
# import libraries
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import geopandas as gpd
from shapely.geometry import shape, Point
import requests
import time
import warnings
import os
import glob

In [ ]:
warnings.filterwarnings('ignore')

## Prepare Data


### Utility Variables


In [ ]:
# relevant area codes
area_codes = [
    'E08000011', 'E08000012', 'E11000002', 'E08000014', 'E08000013',
    'E08000007', 'E06000007', 'E08000010', 'E08000015'
]

# Regions in Merseyside
regions = [
    'Prenton', 'Newton-Le-Willows', 'Birkenhead',
    'Wirral', 'Bootle', 'St Helens', 'Wallasey', 'Southport',
    'Prescot', 'Wigan', 'Widnes', 'Neston', 'Warrington',
    'Ellesmere Port', 'Wilmslow', 'Coniston', 'Stockport', 'Northwood',
    'Crewe', 'Winsford', 'Merseyside', 'Sefton', 'Wirral', 'Liverpool', 'Knowsley'
]

### Utility Functions


In [ ]:
def standardise_column(df, column_name, mapping, remove_values=None, new_dtype='category'):
    """
    Standardise a DataFrame column by applying a mapping, optionally removing specific values, 
    and changing the data type.

    Parameters:
    df (DataFrame): The input DataFrame.
    column_name (str): The name of the column to standardize.
    mapping (dict): A dictionary mapping old values to new values.
    remove_values (list, optional): A list of values to remove from the column. Default is None.
    new_dtype (str, optional): The new data type for the column. Default is 'category'.

    Returns:
    DataFrame: The DataFrame with the standardized column.
    """

    # Optionally remove rows with specific values
    if remove_values:
        df = df[~df[column_name].isin(remove_values)]

    # Apply the mapping to replace old values with new values
    df[column_name] = df[column_name].replace(mapping).astype(new_dtype)

    return df

In [ ]:
def rename_columns(df, new_column_names):
    """
    Rename the columns of a DataFrame using a provided list of new column names.

    Parameters:
    df (DataFrame): The input DataFrame whose columns need to be renamed.
    new_column_names (list): A list of new column names.

    Returns:
    DataFrame: The DataFrame with renamed columns.
    """
    # Check if the number of new column names matches the number of columns in the DataFrame
    if len(new_column_names) != df.shape[1]:
        raise ValueError(
            "The number of new column names must match the number of columns in the DataFrame.")

    # Rename the columns
    df.columns = new_column_names

    return df

### Import


#### Prices Paid Data


In [ ]:
def bulk_lookup_postcodes(postcodes, batch_size=100):
    """
    Perform a bulk lookup for postcodes using the postcodes.io API.

    Parameters:
    postcodes (list): A list of postcodes to lookup.
    batch_size (int): The number of postcodes to include in each batch (max 100).

    Returns:
    dict: A dictionary mapping postcodes to their respective data, including latitude, longitude, and termination data if applicable.
    """
    url = "https://api.postcodes.io/postcodes"
    headers = {'Content-Type': 'application/json'}
    results = {}

    # Process the postcodes in batches
    for i in range(0, len(postcodes), batch_size):
        batch = postcodes[i:i+batch_size]
        data = {"postcodes": batch}

        response = requests.post(url, json=data, headers=headers)

        if response.status_code == 200:
            response_data = response.json()
            for result in response_data['result']:
                postcode = result['query']
                if result['result'] is not None:
                    results[postcode] = {
                        'longitude': result['result']['longitude'],
                        'latitude': result['result']['latitude'],
                        'is_terminated': False
                    }
                else:
                    # If the postcode is terminated, set to None and mark as terminated
                    results[postcode] = {
                        'longitude': None,
                        'latitude': None,
                        'is_terminated': True
                    }
        else:
            print(
                f"Failed to retrieve data for batch starting at {i}: {response.status_code}")
        time.sleep(0.1)  # To avoid rate limiting

    return results

In [ ]:

def handle_terminated_postcodes(postcode_data):
    """
    Handle postcodes that are marked as terminated by checking the terminated postcode API.

    Parameters:
    postcode_data (dict): A dictionary of postcodes with their data, including termination status.

    Returns:
    dict: Updated dictionary with longitude and latitude for terminated postcodes if available.
    """
    terminated_postcodes = [postcode for postcode,
                            data in postcode_data.items() if data['is_terminated']]

    if not terminated_postcodes:
        return postcode_data  # No terminated postcodes to handle

    # Query terminated postcodes in bulk (max 100 at a time)
    url = "https://api.postcodes.io/terminated_postcodes"
    headers = {'Content-Type': 'application/json'}
    batch_size = 100

    for i in range(0, len(terminated_postcodes), batch_size):
        batch = terminated_postcodes[i:i+batch_size]
        data = {"postcodes": batch}

        response = requests.post(url, json=data, headers=headers)

        if response.status_code == 200:
            response_data = response.json()
            for result in response_data['result']:
                postcode = result['query']
                if result['result'] is not None:
                    postcode_data[postcode]['longitude'] = result['result']['longitude']
                    postcode_data[postcode]['latitude'] = result['result']['latitude']
                else:
                    # If still no data, keep as None
                    print(f"No data found for terminated postcode: {postcode}")
        else:
            print(
                f"Failed to retrieve data for terminated postcodes batch starting at {i}: {response.status_code}")
        time.sleep(0.1)  # To avoid rate limiting

    return postcode_data

In [ ]:
def add_long_lat_columns(df, postcode_column='postcode'):
    """
    Add longitude and latitude columns to the DataFrame based on the postcode column using Bulk Lookup.

    Parameters:
    df (DataFrame): The input DataFrame with a postcode column.
    postcode_column (str): The name of the column containing postcodes. Default is 'postcode'.

    Returns:
    DataFrame: The DataFrame with added 'longitude' and 'latitude' columns.
    """
    postcodes = df[postcode_column].unique().tolist()
    postcode_data = bulk_lookup_postcodes(postcodes)
    postcode_data = handle_terminated_postcodes(postcode_data)

    # Map the longitude and latitude back to the original DataFrame
    df['longitude'] = df[postcode_column].map(
        lambda x: postcode_data[x]['longitude'])
    df['latitude'] = df[postcode_column].map(
        lambda x: postcode_data[x]['latitude'])

    return df

In [ ]:
# define wrangle function for prices paid data
def wrangle_prices_paid(filepath):

    # import file
    df = pd.read_csv(filepath)

    # drop complete duplicates from house_data
    df = df.drop_duplicates()

    # add column names to df
    column_names = [
        'transaction_id', 'price', 'transfer_date',
        'postcode', 'property_type', 'is_old_or_new',
        'property_tenure', 'house_number_or_name', 'unit_number',
        'street', 'locality', 'town', 'district', 'county',
        'ppd_transaction_category', 'record_status_monthly_file_only'
    ]
    df = rename_columns(df, column_names)
    
    # convert transfer date to datetime
    df['transfer_date'] = pd.to_datetime(df['transfer_date'])
    
    # sort data by teansaction date
    df = df.sort_values('transfer_date', ascending=True)
    
    # filter data for freehold transactions in merseyside from 2013 to 2023
    df = df[(df['county'] == 'MERSEYSIDE') & (
        df['transfer_date'].dt.year >= 2013) & (df['transfer_date'].dt.year <= 2023)]

    # convert price to float type
    df['price'] = df['price'].astype(float)
    
    # remove outliers in price by values in the bottom and top 5% of properties
    # low, high = df['price'].quantile([0.05, 0.95])
    # mask_area = df['price'].between(low, high)
    # df = df[mask_area]

    # convert ppd_transaction_category to category type
    df['ppd_transaction_category'] = df['ppd_transaction_category'].astype(
        'category')

    # define mappings for replacement
    property_type_mapping = {'T': 'Terraced', 'D': 'Detached', 'F': 'Flats/Maisonettes',
                             'S': 'Semi-Detached', 'O': 'Other'}
    old_or_new_mapping = {'N': 'Old', 'Y': 'New'}
    property_tenure_mapping = {'F': 'Freehold', 'L': 'Leasehold'}

    # standardize 'property_type' column
    df = standardise_column(df, 'property_type', property_type_mapping)
    # standardize 'is_old_or_new' column
    df = standardise_column(df, 'is_old_or_new', old_or_new_mapping)
    # standardize 'property_tenure' column and remove rows with 'U' before standardising
    df = standardise_column(df, 'property_tenure',
                            property_tenure_mapping, remove_values=['U'])

    # convert capital case columns to title case
    df['town'] = df['town'].str.title()
    df['district'] = df['district'].str.title()
    df['county'] = df['county'].str.title()
    
    # exclude rows with null postcode values
    df = df[~df['postcode'].isnull()]
    
    # created latitude and longitude columns
    df = add_long_lat_columns(df)

    # drop redundant columns
    df.drop(columns=['house_number_or_name', 'unit_number', 'locality',
                     'street', 'record_status_monthly_file_only', 'postcode'],
            inplace=True
            )

    return df

In [ ]:
# Wrangle the prices paid data
prices_paid_data_df = wrangle_prices_paid(
    filepath="raw_data/prices_paid.csv")

# Display the first few rows to verify
prices_paid_data_df.head()

#### Employment Data


In [ ]:
# define wrangle function for employment data
def wrangle_employment(filepath):

    # import file
    df = pd.read_csv(filepath)

    # add column names to df
    column_names = ['area_code', 'area_name', 'year', 'employment_rate',
                    'confidence_interval_lower', 'confidence_interval_upper']
    df = rename_columns(df, column_names)
    
    # filter for employment rate from 2013 to 2023
    df = df[(df['year'] >= 2013) & (df['year'] <= 2023)]

    # relevant columns
    relevant_columns = ['area_code', 'area_name', 'year', 'employment_rate']
    # select relevant columns
    df = df[relevant_columns]

    # filter for relevant area codes and names
    df = df[(df['area_name'].isin(
        regions)) | (df['area_code'].isin(
            area_codes))].sort_values('year')
    
    # Convert the year column to string, then to datetime
    df['year'] = df['year'].astype(str)
    df['year'] = pd.to_datetime(
        df['year'], format='%Y')
    # to keep only the year part
    df['year'] = df['year'].dt.year
        
    # Convert employment rate from percentatage to rates
    df['employment_rate'] = df['employment_rate']/100
    
    # aggregate the data
    df = df.groupby(['area_code', 'area_name', 'year']).agg({
        'employment_rate': 'mean'
    }).reset_index()
    
    # drop rows with any missing values
    df = df.dropna()
    
    return df

In [ ]:
# Wrangle the employment data
employment_data_df = wrangle_employment(filepath="raw_data/employment_data.csv")

# Display the first few rows to verify
employment_data_df.head()

#### UK HPI Data

In [ ]:
# define wrangle function for hpi data
def wrangle_hpi(filepath):

    # import file
    df = pd.read_csv(filepath)
    
    # select relevant columns
    relevant_columns = [
        'Date', 'RegionName', 'AreaCode', 'AveragePrice', 'Index', '1m%Change', '12m%Change', 'SalesVolume'
    ]
    df = df[relevant_columns]
    
    # change column names
    column_names = ['date', 'region_name', 'area_code', 'average_price',
                    'index', '1m%_change', '12m%_change', 'sales_volume']
    df = rename_columns(df, column_names)
    
    # change date to dattime instead of object
    df['date'] = pd.to_datetime(df['date'])
    
    # Filter for the date range
    df = df[(df['date'] >= '2013-01-01') & (
        df['date'] <= '2023-12-31')]
    
    # filter for relevant area codes and names
    df = df[(df['region_name'].isin(
        regions)) | (df['area_code'].isin(
            area_codes))].sort_values('date')
    
    # drop complete duplicates
    df = df.drop_duplicates()
    
    # drop rows with any missing values
    df = df.dropna()
    
    return df

In [ ]:
# Wrangle the HPI data
hpi_data_df = wrangle_hpi(filepath="raw_data/uk_hpi.csv")

# Display the first few rows to verify
hpi_data_df.head()

#### Income Data

In [ ]:
# define wrangle function for hpi data
def wrangle_income(filepath):

    # import file
    df = pd.read_csv(filepath)
    
    # change column names
    column_names = ['area_code', 'area_name', 'year',
                        'gross_median_weekly_pay', 'confidence_interval_lower', 'confidence_interval_upper']
    df = rename_columns(df, column_names)
    

    # relevant columns
    relevant_columns = ['area_code', 'area_name',
                        'year', 'gross_median_weekly_pay']
    # select relevant columns
    df = df[relevant_columns]
    
    
    # Convert the year column to string, then to datetime
    df['year'] = df['year'].astype(str)
    df['year'] = pd.to_datetime(
        df['year'], format='%Y')

    # to keep only the year part
    df['year'] = df['year'].dt.year

    # filter for relevant area codes and names
    df = df[(df['area_name'].isin(
        regions)) | (df['area_code'].isin(
            area_codes))].sort_values('year')
    
    # aggregate the data
    df = df.groupby(['area_code', 'area_name', 'year']).agg({
        'gross_median_weekly_pay': 'mean'
    }).reset_index()

    # drop rows with any missing values
    df = df.dropna()
    
    return df

In [ ]:
# Wrangle the income data
income_data_df = wrangle_income(filepath= "raw_data/income_data.csv")

# Display the first few rows to verify
income_data_df.head()

#### Crime Data

In [ ]:
def merge_csv_files_in_folder(folder_path, pattern='*.csv'):
    """
    Merge all CSV files in a given folder into a single DataFrame.

    Parameters:
    folder_path (str): The path to the folder containing the CSV files.
    pattern (str): The pattern to match CSV files in the folder. Default is '*.csv'.

    Returns:
    DataFrame: A merged DataFrame containing all the data from the CSV files.
    """
    # Initialize an empty list to store DataFrames
    df_list = []

    # Get a list of all CSV files in the folder that match the pattern
    csv_files = glob.glob(os.path.join(folder_path, pattern))

    # Iterate over the list of files and read each one into a DataFrame
    for file in csv_files:
        df = pd.read_csv(file)
        df_list.append(df)

    # Concatenate all DataFrames into a single DataFrame
    merged_df = pd.concat(df_list, ignore_index=True)

    return merged_df

merged_crime_data = merge_csv_files_in_folder('raw_data/merseyside_csv_files')

# Display the first few rows to verify the merged data
merged_crime_data.head()

In [ ]:
def wrangle_crime_data(df):
    """
    Cleans and processes crime data by selecting relevant columns, 
    converting the 'Month' to datetime, and sorting by 'Month'.
    
    Parameters:
    df (DataFrame): The input DataFrame containing raw crime data.
    
    Returns:
    DataFrame: A cleaned DataFrame containing only relevant columns.
    """
    # Select relevant columns
    df = df[['Month', 'Longitude', 'Latitude', 'Crime type']]

    # Convert 'Month' to datetime
    df['Month'] = pd.to_datetime(df['Month'], format='%Y-%m').dt.to_period('M')

    # Sort the DataFrame by 'Month'
    df = df.sort_values(by='Month', ascending=True)

    # Optionally, rename the columns for clarity
    df = df.rename(
        columns={'Month': 'crime_date', 'Crime type': 'crime', 'Longitude': 'long', 'Latitude': 'lat'})

    return df

crime_data_df = wrangle_crime_data(merged_crime_data)

# Display the first few rows to verify the cleaning process
crime_data_df.head()

#### Flood Data

In [ ]:
def fetch_flood_data(api_url="https://environment.data.gov.uk/flood-monitoring/id/floodAreas?&_limit=5000"):
    """
    Fetch flood area data from the Environment Agency API.

    Parameters:
    api_url (str): The API endpoint to fetch flood data. Default is set to the flood areas endpoint.

    Returns:
    list: A list of dictionaries containing flood area data.
    """
    response = requests.get(url=api_url)

    if response.status_code == 200:
        # Convert the response to JSON format and extract relevant data
        flood_data = response.json()
        flood_areas = flood_data.get('items', [])
        return flood_areas
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return []


In [ ]:
def prepare_flood_data(flood_areas):
    """
    Prepare and clean flood area data by extracting relevant fields and converting to a DataFrame.

    Parameters:
    flood_areas (list): A list of dictionaries containing flood area data.

    Returns:
    DataFrame: A DataFrame containing the prepared flood area data.
    """
    flood_areas_list = []
    for area in flood_areas:
        flood_areas_list.append({
            'county': area.get('county'),
            'description': area.get('description'),
            'eaAreaName': area.get('eaAreaName'),
            'lat': area.get('lat'),
            'long': area.get('long'),
            'riverOrSea': area.get('riverOrSea'),
            'polygon': area.get('polygon')
        })

    # Convert to DataFrame
    df = pd.DataFrame(flood_areas_list)

    # Rename columns to more descriptive names
    column_names = ['county', 'text_description',
                    'area_name', 'lat', 'lon', 'water_source', 'polygon']
    df = rename_columns(df, column_names)
    
    df = df[df['area_name'].str.contains(
        'mersey', case=False, na=False)]
    
    return df

In [ ]:
def fetch_geojson(uri):
    """
    Fetch and parse GeoJSON data from a polygon URI.

    Parameters:
    uri (str): The URI pointing to the GeoJSON polygon.

    Returns:
    geometry: A Shapely geometry object created from the GeoJSON data.
    """
    response = requests.get(uri)
    if response.status_code == 200:
        geojson = response.json()
        # Access the 'geometry' from the first feature
        if 'features' in geojson and len(geojson['features']) > 0:
            geometry = geojson['features'][0]['geometry']
            return shape(geometry)  # Convert GeoJSON to Shapely geometry
        else:
            print(f"No features found in GeoJSON data from {uri}")
            return None
    else:
        print(f"Failed to retrieve GeoJSON data from {uri}")
        return None


In [ ]:

def convert_to_geodataframe(flood_areas_df):
    """
    Convert the DataFrame to a GeoDataFrame by applying the GeoJSON fetching function.

    Parameters:
    flood_areas_df (DataFrame): The input DataFrame containing flood area data.

    Returns:
    GeoDataFrame: A GeoDataFrame with Shapely geometries for each flood area.
    """
    # Apply the fetch_geojson function to the 'polygon' column
    flood_areas_df['geometry'] = flood_areas_df['polygon'].apply(fetch_geojson)
    
    flood_areas_df.drop('polygon', inplace = True)

    # Convert the DataFrame to a GeoDataFrame
    flood_areas_gdf = gpd.GeoDataFrame(flood_areas_df, geometry='geometry')

    return flood_areas_gdf


In [ ]:
def wrangle_flood_data():
    """
    Complete wrangling process for flood data, including fetching, cleaning, filtering, and conversion to GeoDataFrame.

    Returns:
    GeoDataFrame: A cleaned and filtered GeoDataFrame ready for spatial analysis.
    """
    # Fetch flood data from the API
    flood_areas = fetch_flood_data()

    # Prepare and clean the data
    df = prepare_flood_data(flood_areas)

    # Convert to a GeoDataFrame
    flood_areas_gdf = convert_to_geodataframe(df)

    return flood_areas_gdf


In [ ]:
# Wrangle the flood data and prepare it for analysis
flood_data_gdf = wrangle_flood_data()

# Display the first few rows to verify
flood_data_gdf.head()

### Merge Data

In [ ]:
# def merge_dataframes(prices_paid_df, employment_df, hpi_df, income_df, crime_df, flood_gdf):
#     """
#     Merge multiple dataframes and a geodataframe on relevant keys and spatial relationships.

#     Parameters:
#     prices_paid_df (DataFrame): Cleaned prices paid data.
#     employment_df (DataFrame): Employment data.
#     hpi_df (DataFrame): House price index (HPI) data.
#     income_df (DataFrame): Income data.
#     crime_df (DataFrame): Crime data.
#     flood_gdf (GeoDataFrame): Flood risk area data.

#     Returns:
#     DataFrame: A merged dataframe combining all relevant data.
#     """
#     # Merge employment data based on town and year
#     prices_paid_df['year'] = prices_paid_df['transfer_date'].dt.year
#     merged_df = pd.merge(prices_paid_df, employment_df, left_on=[
#                         'town', 'year'], right_on=['area_name', 'year'], how='left')

#     # Merge HPI data based on area_code and date (rounding date to nearest month)
#     merged_df = pd.merge(merged_df, hpi_df, left_on=[
#                          'district', 'transfer_date'], right_on=['region_name', 'date'], how='left')

#     # Merge income data based on area_code and year
#     merged_df = pd.merge(merged_df, income_df, left_on=[
#                         'district', 'year'], right_on=['area_name', 'year'], how='left')

#     # Merge crime data based on nearest location (latitude and longitude)
#     # For this, we'll need to create spatial points from the latitude and longitude
#     prices_paid_gdf = gpd.GeoDataFrame(merged_df, geometry=gpd.points_from_xy(
#         merged_df.longitude, merged_df.latitude))
#     crime_gdf = gpd.GeoDataFrame(crime_df, geometry=gpd.points_from_xy(
#         crime_df.longitude, crime_df.latitude))

#     # Spatial join to find the nearest crime data points
#     merged_gdf = gpd.sjoin_nearest(
#         prices_paid_gdf, crime_gdf, how='left', distance_col="crime_distance")

#     # Merge flood risk area data based on spatial join
#     merged_gdf = gpd.sjoin(merged_gdf, flood_gdf, how='left', op='intersects')

#     # Convert the final GeoDataFrame back to a DataFrame if needed
#     final_df = pd.DataFrame(merged_gdf.drop(columns='geometry'))

#     return final_df


# # Example usage
# merged_data = merge_dataframes(
#     prices_paid_df, employment_df, hpi_df, income_df, crime_df, flood_gdf)

# # Display the first few rows of the merged dataframe
# print(merged_data.head())

In [ ]:
prices_copy = prices_paid_data_df.copy()
employment_copy = employment_data_df.copy()
income_copy = income_data_df.copy()
flood_copy = flood_data_gdf.copy()
hpi_copy = hpi_data_df.copy()
crime_copy = crime_data_df.copy()

In [ ]:
flood_copy.head()

In [ ]:
# Merge employment data based on area_code and year
prices_copy['year'] = prices_copy['transfer_date'].dt.year
# Merge using district instead of town
merged_prices_employment_copy = pd.merge(
    prices_copy,
    employment_copy,
    left_on=['district', 'year'],
    right_on=['area_name', 'year'],
    how='left'
)

# Forward fill any missing values in the merged dataframe
merged_prices_employment_copy.fillna(method='ffill', inplace=True)

# Display the first few rows to verify
merged_prices_employment_copy.head()

In [ ]:
# Round transfer_date to the nearest month to match HPI data
merged_prices_employment_copy['transfer_date_month'] = merged_prices_employment_copy['transfer_date'].dt.to_period(
    'M')

# Convert HPI date to the same period format for merging
hpi_copy['date_month'] = pd.to_datetime(hpi_copy['date']).dt.to_period('M')

# Merge the dataframes
merged_prices_employment_hpi_copy = pd.merge(
    merged_prices_employment_copy,
    hpi_copy,
    left_on=['district', 'transfer_date_month'],
    right_on=['region_name', 'date_month'],
    how='left'
)

# Forward fill any missing values in the merged dataframe
merged_prices_employment_hpi_copy.fillna(method='ffill', inplace=True)

# Drop the temporary 'transfer_date_month' and 'date_month' columns if needed
merged_prices_employment_hpi_copy.drop(
    columns=['transfer_date_month', 'date_month'], inplace=True)

# Display the first few rows to verify the merge
merged_prices_employment_hpi_copy.head()

In [ ]:
# Assuming merged_prices_employment_hpi_copy is the DataFrame after merging prices, employment, and HPI data

# Merge the dataframes based on district (area_name) and year
merged_prices_employment_hpi_income_copy = pd.merge(
    merged_prices_employment_hpi_copy,
    income_copy,
    left_on=['district', 'year'],
    right_on=['area_name', 'year'],
    how='left'
)

# Forward fill any missing values in the merged dataframe
merged_prices_employment_hpi_income_copy.fillna(method='ffill', inplace=True)

# Display the first few rows to verify the merge
merged_prices_employment_hpi_income_copy.head()

In [ ]:
def merge_with_crime_data(main_df, crime_df):
    """
    Merge the combined property and economic data with crime data using spatial and temporal joins.

    Parameters:
    main_df (DataFrame): The DataFrame containing merged property, employment, HPI, and income data.
    crime_df (DataFrame): The cleaned crime data DataFrame.

    Returns:
    DataFrame: A merged DataFrame with crime data added.
    """
    # Convert the main DataFrame to a GeoDataFrame
    main_gdf = gpd.GeoDataFrame(
        main_df,
        geometry=gpd.points_from_xy(main_df.longitude, main_df.latitude)
    )

    # Convert the crime DataFrame to a GeoDataFrame
    crime_gdf = gpd.GeoDataFrame(
        crime_df,
        geometry=gpd.points_from_xy(crime_df.long, crime_df.lat)
    )

    # Perform spatial join (nearest) and filter by date
    merged_gdf = gpd.sjoin_nearest(
        main_gdf, crime_gdf, how='left', distance_col='crime_distance')

    # Filter for the same date (month)
    merged_gdf = merged_gdf[merged_gdf['transfer_date'].dt.to_period(
        'M') == merged_gdf['crime_date']]

    # Convert back to a DataFrame if needed
    final_df = pd.DataFrame(merged_gdf.drop(columns='geometry'))

    return final_df


# Example usage
merged_prices_employment_hpi_income_crime_copy = merge_with_crime_data(
    merged_prices_employment_hpi_income_copy, crime_copy)

# Display the first few rows to verify the merge
merged_prices_employment_hpi_income_crime_copy.head()

In [ ]:
merged_prices_employment_hpi_income_crime_copy.info()

In [ ]:
def merge_with_flood_data(main_df, flood_gdf):
    """
    Merge the combined property and economic data with flood data using a spatial join.

    Parameters:
    main_df (DataFrame): The DataFrame containing merged property, employment, HPI, income, and crime data.
    flood_gdf (GeoDataFrame): The GeoDataFrame containing flood risk area data.

    Returns:
    DataFrame: A merged DataFrame with flood risk data added.
    """
    # Convert the main DataFrame to a GeoDataFrame, setting the CRS to WGS84 (EPSG:4326)
    main_gdf = gpd.GeoDataFrame(
        main_df,
        geometry=gpd.points_from_xy(main_df.longitude, main_df.latitude),
        # Assuming WGS84 coordinate system, which is common for latitude and longitude
        crs="EPSG:4326"
    )

    # Check and set CRS for the flood GeoDataFrame if not already set
    if flood_gdf.crs is None:
        flood_gdf.set_crs("EPSG:4326", inplace=True)

    # Ensure the flood GeoDataFrame has the same CRS as the main GeoDataFrame
    flood_gdf = flood_gdf.to_crs(main_gdf.crs)

    # Remove 'index_right' if it already exists in the main_gdf
    if 'index_right' in main_gdf.columns:
        main_gdf = main_gdf.drop(columns=['index_right'])

    # Perform a spatial join to add flood risk data to the main DataFrame
    merged_gdf = gpd.sjoin(main_gdf, flood_gdf,
                           how="left", predicate="intersects")

    # Convert back to a regular DataFrame if needed
    final_df = pd.DataFrame(merged_gdf.drop(columns='geometry'))

    return final_df


# Example usage
final_merged_data = merge_with_flood_data(
    merged_prices_employment_hpi_income_crime_copy, flood_data_gdf)

# Display the first few rows to verify the merge
final_merged_data.head()

In [ ]:
final_merged_data.info()

In [ ]:
final_merged_data.shape